In [ ]:
import docker

a = %pwd
wd = (a.rsplit('/',2))[0]

REF_SEQ='Sars_cov_2.ASM985889v3.dna.toplevel.fa'
SEQ_IDS={'ERR4157959': 'portuguese', 'ERR4395294': 'spanish'}
SEQ = " ".join(SEQ_IDS.keys())

D_MEME='quay.io/biocontainers/meme:5.3.0--py37pl526h072abfd_0'

WORK_HOST_DIRECTORY=wd
WORK_CONT_DIRECTORY='/opt/project'

In [ ]:
client = docker.from_env()
bwa_docker_cont = client.containers.run(
    image=D_MEME,
    volumes={WORK_HOST_DIRECTORY: {'bind': WORK_CONT_DIRECTORY,'mode': 'rw'}},
    working_dir= WORK_CONT_DIRECTORY+'/data/mottif',
    command='{0}/scripts_bash/meme.sh {1} "{2}"'.format(WORK_CONT_DIRECTORY, REF_SEQ, SEQ))


In [ ]:
from Bio.motifs import meme
with open('meme.txt') as f:
    record = motifs.parse(f, 'MEME')